In [1]:
import cv2 as cv
import numpy as np
from PIL import Image
from copy import deepcopy

In [2]:
path = "Tests\\resistor_iluminado_crop.png"

In [12]:
def get_similar_colors(image, hsv, thresh = 10):
    pos = list()
    for i in range(len(image)):
        for j in range(len(image[0])):
            if abs(int(hsv[0]) - int(image[i][j][0])) < thresh and abs(int(hsv[2]) - int(image[i][j][2])) < 50: # and abs(int(hsv[1]) - int(image[i][j][1])) < thresh and 
                pos.append([i,j])
    return pos

def get_average_color(pixels):
    somme = [0]*len(pixels[0])
    for i,j,k in pixels:
        somme[0] += i
        somme[1] += j
        somme[2] += k
    avg = np.array(somme) / len(pixels)
    return list(avg)

img = cv.imread(path)
D, I = 4, 150
img = cv.bilateralFilter(img, D, I, I)
img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
imgc = deepcopy(img)
mod = list()  # list of modified positions
for i in range(len(img)):
        for j in range(len(img[0])):
            if [i,j] not in mod:
                pos = get_similar_colors(img, img[i][j])
                mod += pos
                colors = [img[x][y] for x,y in pos]
                avg = get_average_color(colors)
                for x,y in pos:
                    imgc[x][y] = avg


In [13]:
Image.fromarray(cv.cvtColor(imgc, cv.COLOR_HSV2RGB)).save("Tests\\resistor_iluminado_crop_hsv_10_50.png")

In [55]:
from copy import deepcopy
from PIL import Image
import cv2 as cv
import numpy as np
from numpy.linalg import norm

def dot_product(v1, v2):
    return sum(int(a)*int(b) for a,b in zip(v1, v2))

def vect_angle(color1, color2):
    dp = dot_product(color1, color2)/(norm(color1)*norm(color2))
    if dp > 1:
        dp = 1.0
    elif dp < -1:
        dp = -1.0
    return np.arccos(dp)  # in radians

def bilateral_filter(img):
    D, I = 4, 150
    target_width = 150
    scale = target_width/img.shape[1]
    #
    fimg = cv.bilateralFilter(img, D, I, I)
    width = fimg.shape[1]*scale
    height = fimg.shape[0]*scale
    fimg = cv.resize(fimg, (int(width), int(height)))
    return fimg

class Contour:
    def __init__(self, points=[]):
        self.points = list(points)
    # Adds data and/or updates Contour boundaries
    def update(self, points=[]):
        self.points.append(i for i in points)
    # Calculates and returns the sub-boundaries in self.points
    def get_boundaries(self):
        # Returns neighbors to a point dist-pixels away
        def get_neighbors(point, dist=3):
            n = list()
            for i in range(-(dist//2),1+dist//2):
                for j in range(-(dist//2),1+dist//2):
                    n.append(list(np.add(point, (i,j))))
            return n
        #
        def in_boundary(point, boundary):
            if point >= boundary[0] and point < boundary[1]:
                return True
            else:
                return False
        # Iteratively finds sub-boundaries
        subb = []
        points = deepcopy(self.points)
        for point in self.points:
            contained = False
            index = -1
            for i in range(len(subb)):
                if not contained and in_boundary(point, subb[i]):
                    contained = True
                    index = i
                    break
            if not contained:
                subb.append([point, point])
            #
            neighbors = [n for n in get_neighbors(point) if (list(n) in self.points or tuple(n) in self.points)]
            # Updates boundary with top-left-most and bottom-right-most points in its proximity
            subb[index][0] = min(list(min(neighbors)), list(subb[index][0]))
            subb[index][1] = max(list(max(neighbors)), list(subb[index][1]))
        return subb            
    # Returns the index-th boundary
    def __getitem__(self, index):
        return self.points[index]


class Color:
    def __init__(self, image, pos, thresh = [10,255,255]):
        self.image = image
        self.color = image[pos[0]][pos[1]]
        self.coords = [pos]  # List of the coordinates of the pixels that have the 'same' color
        self.thresh = thresh
    # Overloads the equality operator (==)
    def __eq__(self, pos):
        c = self.image[pos[0]][pos[1]]
        if abs(int(self.color[0]) - int(c[0])) <= self.thresh[0] and abs(int(self.color[1]) - int(c[1])) <= self.thresh[1] and abs(int(self.color[2]) - int(c[2])) <= self.thresh[2]:
            return True
        else:
            return False
    # Overloads the contains operator (in)
    def __contains__(self, pos):
        if pos in self.coords:
            return True
        return False
        #return self.__eq__(pos)
    # Adds a color to the group
    def append(self, pos):
        somm = np.array(self.color).astype(int) * len(self.coords)
        if pos not in self.coords:
            self.coords.append(pos)
        somm += self.image[pos[0]][pos[1]]
        self.color = list(int(i) for i in somm/len(self.coords))

class Group:
    def __init__(self, objs = []):
        self.objects = objs
    # Adds an object to the group
    def insert(self, obj):
        self.objects.append(obj)
    # Checks if an element is in any of its subgroups
    def __contains__(self, elem):
        for col in self.objects:
            if elem == col:
                return True
        return False
    # Adds an element to one of its subgroups
    def append(self, elem):
        for col in self.objects:
            if elem == col:
                col.append(elem)
                return

def find_color_pools(im, color_space=cv.COLOR_BGR2HSV):
    """
    """
    img = cv.cvtColor(im, color_space)
    groups = Group()
    for i in range(len(img)):
        for j in range(len(img[i])):
            if (i,j) not in groups:
                cg = Color(img, (i,j))
                groups.insert(cg)
            else:
                groups.append([i,j])
    return groups

file = "Tests\\res.png"
img = cv.imread(file)
img = bilateral_filter(img)
groups = find_color_pools(img)
imgcp = deepcopy(img)
for obj in groups.objects:
    for point in obj.coords:
        imgcp[point[0]][point[1]] = obj.color
#
Image.fromarray(cv.cvtColor(imgcp, cv.COLOR_HSV2RGB)).save("Tests\\res_hsv_cg.png")
